In [1]:
source = "Recommendations";

In [2]:
using NBInclude
@nbinclude("../ProductionAlphas/Alpha.ipynb");

In [3]:
ENV["COLUMNS"] = 999999;
#ENV["LINES"] = 100;

In [4]:
anime = DataFrame(CSV.File("../../data/raw_data/anime.csv"))
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [5]:
predictions = read_recommendee_alpha("CombineSignals").rating
rating_df = DataFrame("uid" => 0:length(predictions)-1, "rating" => predictions);

In [6]:
df = get_recommendee_list()
@debug rmse(df.rating, predictions[df.item])
@debug mae(df.rating, predictions[df.item])
@debug r2(df.rating, predictions[df.item])
rating_df.rating[df.item] .= 0;

[ Debug: 20220102 22:23:08 1.3676658651157594
[ Debug: 20220102 22:23:08 1.0341588048722163
[ Debug: 20220102 22:23:08 0.35714737801661034


In [7]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");
rec_df = rec_df[:, ["anime_id", "title", "genres", "medium", "rating"]];
filter(x -> x.medium == "tv", sort!(rec_df, :rating, rev = true))

,anime_id,title,genres,medium,rating
,Int64,String,String,String7,Float64
1,45576,Mushoku Tensei: Isekai Ittara Honki Dasu Part 2,"['Drama', 'Ecchi', 'Fantasy']",tv,7.8878
2,32843,Senki Zesshou Symphogear XV,"['Action', 'Music', 'Sci-Fi']",tv,7.85734
3,2921,Ashita no Joe 2,"['Action', 'Drama', 'Shounen', 'Slice of Life', 'Sports']",tv,7.69286
4,42587,Yuuki Yuuna wa Yuusha de Aru: Dai Mankai no Shou,"['Drama', 'Fantasy', 'Slice of Life']",tv,7.68312
5,38524,Shingeki no Kyojin Season 3 Part 2,"['Action', 'Drama', 'Fantasy', 'Military', 'Mystery', 'Shounen', 'Super Power']",tv,7.65835
6,40028,Shingeki no Kyojin: The Final Season,"['Action', 'Drama', 'Fantasy', 'Military', 'Mystery', 'Shounen', 'Super Power']",tv,7.54773
7,41169,Love Live! Superstar!!,"['Music', 'School', 'Slice of Life']",tv,7.46555
8,19111,Love Live! School Idol Project 2nd Season,"['Music', 'School', 'Slice of Life']",tv,7.44008
9,11061,Hunter x Hunter (2011),"['Action', 'Adventure', 'Fantasy', 'Shounen', 'Super Power']",tv,7.43426


In [7]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");
rec_df = rec_df[:, ["anime_id", "title", "genres", "medium", "rating"]];
filter(x -> x.medium == "tv", sort!(rec_df, :rating, rev = true))

,anime_id,title,genres,medium,rating
,Int64,String,String,String7,Float64
1,32843,Senki Zesshou Symphogear XV,"['Action', 'Music', 'Sci-Fi']",tv,7.85724
2,42587,Yuuki Yuuna wa Yuusha de Aru: Dai Mankai no Shou,"['Drama', 'Fantasy', 'Slice of Life']",tv,7.75105
3,45576,Mushoku Tensei: Isekai Ittara Honki Dasu Part 2,"['Drama', 'Ecchi', 'Fantasy']",tv,7.70775
4,38524,Shingeki no Kyojin Season 3 Part 2,"['Action', 'Drama', 'Fantasy', 'Military', 'Mystery', 'Shounen', 'Super Power']",tv,7.70673
5,2921,Ashita no Joe 2,"['Action', 'Drama', 'Shounen', 'Slice of Life', 'Sports']",tv,7.7061
6,40028,Shingeki no Kyojin: The Final Season,"['Action', 'Drama', 'Fantasy', 'Military', 'Mystery', 'Shounen', 'Super Power']",tv,7.70556
7,19111,Love Live! School Idol Project 2nd Season,"['Music', 'School', 'Slice of Life']",tv,7.53298
8,11061,Hunter x Hunter (2011),"['Action', 'Adventure', 'Fantasy', 'Shounen', 'Super Power']",tv,7.48605
9,3750,Maria-sama ga Miteru 4th,"['Girls Love', 'School', 'Shoujo', 'Slice of Life']",tv,7.40074


In [8]:
sort!(rec_df, :rating, rev = true)

,anime_id,title,genres,medium,rating
,Int64,String,String,String7,Float64
1,34376,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 3 - Yakusoku,"['Drama', 'Fantasy', 'Slice of Life']",movie,9.22151
2,34375,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 2 - Tamashii,"['Drama', 'Fantasy', 'Slice of Life']",movie,8.95603
3,34374,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 1 - Tomodachi,"['Drama', 'Fantasy', 'Slice of Life']",movie,8.69906
4,11979,Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari,"['Drama', 'Psychological', 'Suspense']",movie,8.47786
5,11977,Mahou Shoujo Madoka★Magica Movie 1: Hajimari no Monogatari,"['Drama', 'Psychological', 'Suspense']",movie,8.43559
6,609,Saishuu Heiki Kanojo: Another Love Song,"['Drama', 'Romance', 'Sci-Fi']",ova,8.35309
7,41674,Aria the Crepuscolo,"['Fantasy', 'Sci-Fi', 'Shounen', 'Slice of Life']",movie,8.2125
8,29831,Tamayura: Sotsugyou Shashin Part 4 - Ashita,"['Comedy', 'Drama', 'Slice of Life']",movie,8.02053
9,32843,Senki Zesshou Symphogear XV,"['Action', 'Music', 'Sci-Fi']",tv,7.85724
